In [ ]:
#구글 드라이브 연결
from google.colab import drive
drive.mount('/content/gdrive')

#코랩 환경 경로 설정 -> 자신에게 맞는 경로로 설정해주시면 됩니다
DATA_PATH = '/content/gdrive/MyDrive/미세먼지예측공모전/dataset/'

Mounted at /content/gdrive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 0s (37.0 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 122518 files and dir

In [ ]:
import pandas as pd
import gc
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

from sklearn.preprocessing import LabelEncoder

import platform
# 차트에서 한글 출력을 위한 설정
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()

    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)


import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64')

device = torch.device('cuda')


gc.collect()

23

In [ ]:
train = pd.read_csv(DATA_PATH + '/train.csv', encoding = "utf-8")
test = pd.read_csv(DATA_PATH + '/test.csv', encoding = "utf-8")

In [ ]:
sub = pd.read_csv(DATA_PATH + '/answer_sample.csv')

___

In [ ]:
train = train[~train['일시'].str.startswith('02-29')].reset_index(drop=True)

In [ ]:
train['연도'] = train['연도'].astype(str) + '_' + train['측정소']

In [ ]:
test['연도'] = test['연도'].astype(str) + '_' + test['측정소']

In [ ]:
train = train.drop(columns="측정소")
test = test.drop(columns="측정소")

In [ ]:
train = train[['연도', '일시', 'PM2.5', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)']]
test = test[['연도', '일시', 'PM2.5', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)']]

In [ ]:
train

,연도,일시,PM2.5,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0_공주,01-01 00:00,0.056,0.173776,0.201944,0.023018,0.0,0.828
1,0_공주,01-01 01:00,0.060,0.176935,0.168611,0.030691,0.0,0.831
2,0_공주,01-01 02:00,0.068,0.180095,0.087222,0.033248,0.0,0.784
3,0_공주,01-01 03:00,0.060,0.178515,0.087222,0.025575,0.0,0.745
4,0_공주,01-01 04:00,0.068,0.164297,0.113889,0.020460,0.0,0.750
...,...,...,...,...,...,...,...,...
595675,3_홍성읍,12-31 19:00,0.060,0.273302,0.832222,0.086957,0.0,0.671
595676,3_홍성읍,12-31 20:00,0.052,0.271722,0.831667,0.043478,0.0,0.692
595677,3_홍성읍,12-31 21:00,0.044,0.268562,0.832500,0.066496,0.0,0.706
595678,3_홍성읍,12-31 22:00,0.052,0.262243,0.866944,0.043478,0.0,0.725


In [ ]:
# train 데이터의 '연도' 컬럼을 factorize() 함수를 사용하여 정수로 변환
train['연도'], train_uni = pd.factorize(train['연도'])

In [ ]:
test['연도'], test_uni = pd.factorize(test['연도'])

## Label Encoding

In [ ]:
# Target 값 정규화
mini=train.iloc[:,2].min()
size=train.iloc[:,2].max()-train.iloc[:,2].min()
train.iloc[:,2]=(train.iloc[:,2]-mini)/size

In [ ]:
train #17개의 측정소 X 365 일 24시간 =148920

,연도,일시,PM2.5,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,0.056,0.173776,0.201944,0.023018,0.0,0.828
1,0,01-01 01:00,0.060,0.176935,0.168611,0.030691,0.0,0.831
2,0,01-01 02:00,0.068,0.180095,0.087222,0.033248,0.0,0.784
3,0,01-01 03:00,0.060,0.178515,0.087222,0.025575,0.0,0.745
4,0,01-01 04:00,0.068,0.164297,0.113889,0.020460,0.0,0.750
...,...,...,...,...,...,...,...,...
595675,67,12-31 19:00,0.060,0.273302,0.832222,0.086957,0.0,0.671
595676,67,12-31 20:00,0.052,0.271722,0.831667,0.043478,0.0,0.692
595677,67,12-31 21:00,0.044,0.268562,0.832500,0.066496,0.0,0.706
595678,67,12-31 22:00,0.052,0.262243,0.866944,0.043478,0.0,0.725


In [ ]:
test = test.dropna()

In [ ]:
test #17개의 측정소 X 128일 24시간 =52224

,연도,일시,PM2.5,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,0.060,0.244866,0.123333,0.038363,0.0,0.647
1,0,01-01 01:00,0.064,0.232227,0.167778,0.033248,0.0,0.648
2,0,01-01 02:00,0.072,0.206951,0.000000,0.002558,0.0,0.734
3,0,01-01 03:00,0.064,0.199052,0.000000,0.002558,0.0,0.753
4,0,01-01 04:00,0.056,0.189573,0.000000,0.002558,0.0,0.795
...,...,...,...,...,...,...,...,...
130483,16,11-13 19:00,0.160,0.477093,0.716944,0.030691,0.0,0.911
130484,16,11-13 20:00,0.172,0.494471,0.575833,0.012788,0.0,0.933
130485,16,11-13 21:00,0.160,0.458136,0.772222,0.043478,0.0,0.982
130486,16,11-13 22:00,0.156,0.483412,0.000000,0.007673,0.0,0.982


## DataLoader

In [ ]:
sub

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,NaN
1,4,01-03 01:00,공주,NaN
2,4,01-03 02:00,공주,NaN
3,4,01-03 03:00,공주,NaN
4,4,01-03 04:00,공주,NaN
...,...,...,...,...
78331,4,11-16 19:00,홍성읍,NaN
78332,4,11-16 20:00,홍성읍,NaN
78333,4,11-16 21:00,홍성읍,NaN
78334,4,11-16 22:00,홍성읍,NaN


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
input_window = 48
output_window = 72 #3(일)*24(시간)
window = 12 #window는 12시간 마다는 12시간 마다
num_features = 6
num_year = 68
end_=192
lstm_units=32
dropout=0.2
EPOCH=30
BATCH_SIZE=128

In [ ]:
train

,연도,일시,PM2.5,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,0.056,0.173776,0.201944,0.023018,0.0,0.828
1,0,01-01 01:00,0.060,0.176935,0.168611,0.030691,0.0,0.831
2,0,01-01 02:00,0.068,0.180095,0.087222,0.033248,0.0,0.784
3,0,01-01 03:00,0.060,0.178515,0.087222,0.025575,0.0,0.745
4,0,01-01 04:00,0.068,0.164297,0.113889,0.020460,0.0,0.750
...,...,...,...,...,...,...,...,...
595675,67,12-31 19:00,0.060,0.273302,0.832222,0.086957,0.0,0.671
595676,67,12-31 20:00,0.052,0.271722,0.831667,0.043478,0.0,0.692
595677,67,12-31 21:00,0.044,0.268562,0.832500,0.066496,0.0,0.706
595678,67,12-31 22:00,0.052,0.262243,0.866944,0.043478,0.0,0.725


In [ ]:
#train을 tensor로 변경
train_x=tf.reshape(train.iloc[:,2:].values, [num_year, 24*365, num_features])
print(f'train_x.shape:{train_x.shape}')

train_x.shape:(68, 8760, 6)


In [ ]:
#train_window_x np.zeros를 만듬 (68, 365, 48, 1)
train_window_x= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, input_window, num_features))
train_window_y= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, output_window, 1))
print(f'train_window_x.shape:{train_window_x.shape}')
print(f'train_window_y.shape:{train_window_y.shape}')

train_window_x.shape:(68, 720, 48, 6)
train_window_y.shape:(68, 720, 72, 1)


In [ ]:
#train_window_x에 train값 채워넣기
for example in range(train_x.shape[0]):

    for start in range(0, train_x.shape[1]-(input_window+output_window), window):
        end=start+input_window
        train_window_x[example, start//window, :] = train_x[example, start: end               , :]
        train_window_y[example, start//window, :] = train_x[example, end  : end+ output_window, :1]

In [ ]:
#new_train_x, reshape통해 lstm에 알맞은 형태로 집어넣기
new_train_x=tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y=tf.reshape(train_window_y, [-1, output_window,1])
print(f'new_train_x.shape:{new_train_x.shape}')
print(f'new_train_y.shape:{new_train_y.shape}')

new_train_x.shape:(48960, 48, 6)
new_train_y.shape:(48960, 72, 1)


In [ ]:
#####층 쌓기###########


model=Sequential([
LSTM(lstm_units, return_sequences=False, recurrent_dropout=dropout),
Dense(output_window * num_features, kernel_initializer=tf.initializers.zeros()),
Reshape([output_window, num_features])
])

In [ ]:
#######Compile 구성하기################


model.compile(optimizer='rmsprop', loss='mae', metrics=['mae'])
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0: print('')
        print('.', end='')

#가장 좋은 성능을 낸 val_loss가 적은 model만 남겨 놓았습니다.
save_best_only=tf.keras.callbacks.ModelCheckpoint(filepath="lstm_model.h5", monitor='val_loss', save_best_only=True)


early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

#검증 손실이 10epoch 동안 좋아지지 않으면 학습률을 0.1 배로 재구성하는 명령어입니다.
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

In [ ]:
######################
model.fit(new_train_x, new_train_y, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2, verbose=0,
          callbacks=[PrintDot(), early_stop, save_best_only , reduceLR])

model.summary()


..........
..........
..........Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4992      
                                                                 
 dense (Dense)               (None, 432)               14256     
                                                                 
 reshape (Reshape)           (None, 72, 6)             0         
                                                                 
Total params: 19,248
Trainable params: 19,248
Non-trainable params: 0
_________________________________________________________________


In [ ]:
test

,연도,일시,PM2.5,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,0.060,0.244866,0.123333,0.038363,0.0,0.647
1,0,01-01 01:00,0.064,0.232227,0.167778,0.033248,0.0,0.648
2,0,01-01 02:00,0.072,0.206951,0.000000,0.002558,0.0,0.734
3,0,01-01 03:00,0.064,0.199052,0.000000,0.002558,0.0,0.753
4,0,01-01 04:00,0.056,0.189573,0.000000,0.002558,0.0,0.795
...,...,...,...,...,...,...,...,...
130483,16,11-13 19:00,0.160,0.477093,0.716944,0.030691,0.0,0.911
130484,16,11-13 20:00,0.172,0.494471,0.575833,0.012788,0.0,0.933
130485,16,11-13 21:00,0.160,0.458136,0.772222,0.043478,0.0,0.982
130486,16,11-13 22:00,0.156,0.483412,0.000000,0.007673,0.0,0.982


In [ ]:
len(list(test.일시.unique()))

3072

In [ ]:
#test을 tensor로 변경
test_x=tf.reshape(test.iloc[:,2:].values, [17, 24*128, num_features])
print(f'test_x.shape:{test_x.shape}')

test_x.shape:(17, 3072, 6)


In [ ]:
#test_window_x np.zeros를 만듬 (68, 365, 48, 1)
test_window_x= np.zeros(( test_x.shape[0], (test_x.shape[1]-(input_window + output_window))//window, input_window, num_features))
test_window_y= np.zeros(( test_x.shape[0], (test_x.shape[1]-(input_window + output_window))//window, output_window, 1))
print(f'test_window_x.shape:{test_window_x.shape}')
print(f'test_window_y.shape:{test_window_y.shape}')

test_window_x.shape:(17, 246, 48, 6)
test_window_y.shape:(17, 246, 72, 1)


In [ ]:
#test_window_x에 test값 채워넣기
for example in range(test_x.shape[0]):

    for start in range(0, test_x.shape[1]-(input_window+output_window), window):
        end=start+input_window
        test_window_x[example, start//window, :] = test_x[example, start: end               , :]
        test_window_y[example, start//window, :] = test_x[example, end  : end+ output_window, :1]

In [ ]:
#new_test_x, reshape통해 lstm에 알맞은 형태로 집어넣기
new_test_x=tf.reshape(test_window_x, [-1, input_window, num_features])
new_test_y=tf.reshape(test_window_y, [-1, output_window,1])
print(f'new_test_x.shape:{new_test_x.shape}')
print(f'new_test_y.shape:{new_test_y.shape}')

new_test_x.shape:(4182, 48, 6)
new_test_y.shape:(4182, 72, 1)


In [ ]:
#######################
prediction=np.zeros((17, end_*4, num_features))
new_test_x=test_x

for i in range(end_//output_window):
    start_=i*output_window
    next_=model.predict(new_test_x[ : , -input_window:, :])
    new_test_x = tf.concat([new_test_x, next_], axis=1)
    print(new_test_x.shape)
    prediction[:, start_: start_ + output_window, :]= next_
prediction =prediction *size + mini

1/1 [==============================] - 0s 32ms/step
(17, 3144, 6)
1/1 [==============================] - 0s 33ms/step
(17, 3216, 6)


In [ ]:
sub['PM2.5']=prediction.reshape([-1,1])

In [ ]:
sub['PM2.5']

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,0.125349
1,4,01-03 01:00,공주,0.125349
2,4,01-03 02:00,공주,0.125349
3,4,01-03 03:00,공주,0.125349
4,4,01-03 04:00,공주,0.125349
...,...,...,...,...
78331,4,11-16 19:00,홍성읍,0.000000
78332,4,11-16 20:00,홍성읍,0.000000
78333,4,11-16 21:00,홍성읍,0.000000
78334,4,11-16 22:00,홍성읍,0.000000


In [ ]:
sub.to_csv(DATA_PATH + 'lstm_version_01.csv', encoding='utf-8', index=False)